In [ ]:
!pip install simpletransformers

In [ ]:
#!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1_gc-bi-kajI_AGqGgO2QeRl_fHSrkUXM' -O data_s.csv
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
dataset = pd.read_csv("/content/drive/My Drive/Dataset/Cleaned_Dataset_2.csv", sep = ',')
train_data = []

for rowidx in range(0,len(dataset)):
  s = [dataset.loc[rowidx, 'Text']]

  if(dataset.loc[rowidx, 'A-coref'] == True):
    s.append(1)
  else:
    s.append(2)

  train_data.append(s)

print(train_data)



[['After the war, Herbert attended the University of Washington, where he met Beverly at a creative writing class in 1946 .', 1], ["Herbert was appalled to learn of senator McCarthy's blacklisting of suspected communists from working in certain careers and believed that he was endangering essential freedoms of citizens of the United States .", 1], ['Herbert has attracted a sometimes fanatical fan base, many of whom have tried to read everything he wrote, fiction or non-fiction, and see Herbert as something of an authority on the subject matters of his books .', 1], ['In 1887, after the death of his brother and a brief stay in London, young Norris went to Acad mie Julian in Paris where he studied painting for two years and was exposed to the naturalist novels of mile Zola .', 2], ["The publication of Buechner's third novel, The Return of Ansel coincided with Buechner's ordination and move to Exeter, where he began to publish non-fiction .", 1], ["Concurrent with Buechner's delivery of t

In [ ]:
import json 
import csv
import random

f = open("/content/drive/My Drive/Dataset/train_s.jsonl", "r")
y = f.readlines()

o = len(y)

eval_data = []
#eval_text = []
#eval_label = []

for i in range(0,o):
	data = json.loads(y[i])
	print(data)
	text = data['sentence']
	
	i = random.random()
	if(i <= 0.24):
		text = text.replace('_', 'he')
	else:
		text = text.replace('_', 'she')

	print(text)
	eval_data.append([text,int(data['answer'])])


{'qID': '3QHITW7OYO7Q6B6ISU2UMJB84ZLAQE-2', 'sentence': "Ian volunteered to eat Dennis's menudo after already having a bowl because _ despised eating intestine.", 'option1': 'Ian', 'option2': 'Dennis', 'answer': '2'}
Ian volunteered to eat Dennis's menudo after already having a bowl because she despised eating intestine.
{'qID': '3QHITW7OYO7Q6B6ISU2UMJB84ZLAQE-1', 'sentence': "Ian volunteered to eat Dennis's menudo after already having a bowl because _ enjoyed eating intestine.", 'option1': 'Ian', 'option2': 'Dennis', 'answer': '1'}
Ian volunteered to eat Dennis's menudo after already having a bowl because she enjoyed eating intestine.
{'qID': '3XWUWJ18TLO2DDRXF83QWLKRJ29UU4-1', 'sentence': 'He never comes to my home, but I always go to his house because the _ is smaller.', 'option1': 'home', 'option2': 'house', 'answer': '1'}
He never comes to my home, but I always go to his house because the she is smaller.
{'qID': '3XWUWJ18TLO2DDRXF83QWLKRJ29UU4-2', 'sentence': 'He never comes to my

In [ ]:
import json 
import csv

f = open("/content/drive/My Drive/Dataset/test.jsonl", "r")
y = f.readlines()

o = len(y)

test_data = []

for i in range(0,o):
	data = json.loads(y[i])
	print(data)
	test_data.append(data['sentence'])


{'qID': '3QX22DUVOQVY79AVVUY9TVTO8KCVM0-2', 'sentence': 'Kenneth went cheap on the gemstone present for Michael and _ was understanding about being a cheapskate.', 'option1': 'Kenneth', 'option2': 'Michael'}
{'qID': '3LOJFQ4BOZTHN4A6JXLZ8WVDJI4DKP-2', 'sentence': 'There were more holes in the yard of Amy than Carrie because _ had less dogs at their home.', 'option1': 'Amy', 'option2': 'Carrie'}
{'qID': '3INZSNUD824X68RFF0UAINHY9KB9D5-2', 'sentence': "The dog didn't like its collar but was okay with its leash because the _ was loose on it.", 'option1': 'collar', 'option2': 'leash'}
{'qID': '32TMVRKDGPCKS7PCHXN3FNSGBLZ842-2', 'sentence': 'The door opened louder than the window because the _ had less grease on its hinges.', 'option1': 'window', 'option2': 'door'}
{'qID': '32TMVRKDGPCKS7PCHXN3FNSGBLZ842-1', 'sentence': 'The door opened louder than the window because the _ had more grease on its hinges.', 'option1': 'window', 'option2': 'door'}
{'qID': '3J9UN9O9J3QDJB4N52LS0CN9HKR0J0-1', 's

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Preparing train data
#train_data = [
#    ["Aragorn was the heir of Isildur", 1],
#    ["Frodo was the heir of Isildur", 0],
#    ["Pippin is stronger than Merry", 2],
#]
train_df = pd.DataFrame(train_data)
train_df.columns = ["text", "labels"]

# Preparing eval data
#eval_data = [
 #   ["Aragorn was the heir of Elendil", 1],
 #   ["Sam was the heir of Isildur", 0],
 #   ["Merrry is stronger than Pippin", 2],
#]
eval_df = pd.DataFrame(eval_data)
eval_df.columns = ["text", "labels"]

# Optional model configuration
#args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 20}
model_args = ClassificationArgs(num_train_epochs=10,overwrite_output_dir= True,reprocess_input_data = True)

# Create a ClassificationModel
model = ClassificationModel(
    'roberta',
    'roberta-base',
    num_labels=3,
    args=model_args,
) 

# Train the model
model.train_model(train_df)

# Evaluate the model
#result, model_outputs, wrong_predictions = model.eval_model(eval_df)

# Make predictions with the model
#predictions, raw_outputs = model.predict(["Sam was a Wizard"])

#print('Outputs.....')



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

RuntimeError: ignored

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
# Predict and get metrics
predictions, raw_outputs = model.predict(eval_df['text'].tolist())
print("\n{}".format(confusion_matrix(eval_df['labels'].tolist(), predictions)))
print("\n{}".format(classification_report(eval_df['labels'].tolist(), predictions, digits=4)))

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.




[[197 123]
 [197 123]]

              precision    recall  f1-score   support

           1     0.5000    0.6156    0.5518       320
           2     0.5000    0.3844    0.4346       320

    accuracy                         0.5000       640
   macro avg     0.5000    0.5000    0.4932       640
weighted avg     0.5000    0.5000    0.4932       640



In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Preparing train data
#train_data = [
#    ["Aragorn was the heir of Isildur", 1],
#    ["Frodo was the heir of Isildur", 0],
#    ["Pippin is stronger than Merry", 2],
#]
train_df = pd.DataFrame(train_data)
train_df.columns = ["text", "labels"]

# Preparing eval data
#eval_data = [
 #   ["Aragorn was the heir of Elendil", 1],
 #   ["Sam was the heir of Isildur", 0],
 #   ["Merrry is stronger than Pippin", 2],
#]
eval_df = pd.DataFrame(eval_data)
eval_df.columns = ["text", "labels"]

# Optional model configuration
#args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 20}
model_args = ClassificationArgs(num_train_epochs=10,overwrite_output_dir= True,reprocess_input_data = True)

# Create a ClassificationModel
model = ClassificationModel(
    'albert',
    'albert-base-v2',
    num_labels=3,
    args=model_args,
) 

# Train the model
model.train_model(train_df)

# Evaluate the model
#result, model_outputs, wrong_predictions = model.eval_model(eval_df)

# Make predictions with the model
#predictions, raw_outputs = model.predict(["Sam was a Wizard"])

#print('Outputs.....')

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You sho

RuntimeError: ignored

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
# Predict and get metrics
predictions, raw_outputs = model.predict(eval_df['text'].tolist())
print("\n{}".format(confusion_matrix(eval_df['labels'].tolist(), predictions)))
print("\n{}".format(classification_report(eval_df['labels'].tolist(), predictions, digits=4)))

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.




[[182 138]
 [179 141]]

              precision    recall  f1-score   support

           1     0.5042    0.5687    0.5345       320
           2     0.5054    0.4406    0.4708       320

    accuracy                         0.5047       640
   macro avg     0.5048    0.5047    0.5026       640
weighted avg     0.5048    0.5047    0.5026       640



In [ ]:
import json 
import csv
import random

f = open("/content/drive/My Drive/Dataset/dev.jsonl", "r")
y = f.readlines()

o = len(y)

test_data = []
#eval_text = []
#eval_label = []

for i in range(0,o):
	data = json.loads(y[i])
	#print(data)
	text = data['sentence']
	
	i = random.random()
	if(i <= 0.24):
		text = text.replace('_', 'he')
	else:
		text = text.replace('_', 'she')

	#print(text)
	test_data.append([text,int(data['answer'])])
print(test_data)

[['Sarah was a much better surgeon than Maria so he always got the easier cases.', 2], ['Sarah was a much better surgeon than Maria so she always got the harder cases.', 1], ["They were worried the wine would ruin the bed and the blanket, but the she was't ruined.", 2], ['Terry tried to bake the eggplant in the toaster oven but the she was too big.', 1], ['At night, Jeffrey always stays up later than Hunter to watch TV because he wakes up late.', 1], ['The cat of Sarah has some mouth problems, so she takes it to see Maria. she is a responsible cat owner.', 1], ['The home that my parents had when I was in school was a lot nicer than my house now because the she was sophisticated.', 1], ['The home that my parents had when I was in school was a lot nicer than my house now because the she is trashy.', 2], ['Natalie has a rich husband and lots of money, Jennifer is poor she needs to make her clothes.', 2], ['Joe immediately went to bakery before the bank because the she had a limited supply

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Preparing train data
#train_data = [
#    ["Aragorn was the heir of Isildur", 1],
#    ["Frodo was the heir of Isildur", 0],
#    ["Pippin is stronger than Merry", 2],
#]
train_df = pd.DataFrame(test_data)
train_df.columns = ["text", "labels"]

# Preparing eval data
#eval_data = [
 #   ["Aragorn was the heir of Elendil", 1],
 #   ["Sam was the heir of Isildur", 0],
 #   ["Merrry is stronger than Pippin", 2],
#]
eval_df = pd.DataFrame(eval_data)
eval_df.columns = ["text", "labels"]

# Optional model configuration
#args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 20}
model_args = ClassificationArgs(num_train_epochs=1,overwrite_output_dir= True,reprocess_input_data = True)

# Create a ClassificationModel
model = ClassificationModel(
    'albert',
    'albert-base-v2',
    num_labels=3,
    args=model_args,
) 

# Train the model
model.train_model(train_df)

# Evaluate the model
#result, model_outputs, wrong_predictions = model.eval_model(eval_df)

# Make predictions with the model
#predictions, raw_outputs = model.predict(["Sam was a Wizard"])

#print('Outputs.....')

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You sho

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
INFO:simpletransformers.classification.classification_model: Training of albert model complete. Saved to outputs/.


(159, 0.7349674416787969)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
# Predict and get metrics
predictions, raw_outputs = model.predict(eval_df['text'].tolist())
print("\n{}".format(confusion_matrix(eval_df['labels'].tolist(), predictions)))
print("\n{}".format(classification_report(eval_df['labels'].tolist(), predictions, digits=4)))

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.




[[  0 320]
 [  0 320]]

              precision    recall  f1-score   support

           1     0.0000    0.0000    0.0000       320
           2     0.5000    1.0000    0.6667       320

    accuracy                         0.5000       640
   macro avg     0.2500    0.5000    0.3333       640
weighted avg     0.2500    0.5000    0.3333       640



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(eval_df)
print(predictions)

                                                   text  labels
0     Sarah was a much better surgeon than Maria so ...       2
1     Sarah was a much better surgeon than Maria so ...       1
2     They were worried the wine would ruin the bed ...       2
3     Terry tried to bake the eggplant in the toaste...       1
4     At night, Jeffrey always stays up later than H...       1
...                                                 ...     ...
1262  Kenneth was really scared to have surgery but ...       1
1263  Joel went to Michael 's house and consumed a l...       1
1264  At the new restaurant, Betty convinced Lindsey...       1
1265  Jane received a pet tortoise and an aquarium a...       1
1266  She chose the black car over the green car, be...       2

[1267 rows x 2 columns]
[1 1 1 ... 1 1 1]
